In [1]:
import quasar

In [2]:
circuit = quasar.Circuit()
circuit.H(0).X(1).Y(2).Z(3)
circuit.CX(0,1).CX(2,3)
circuit.CX(0,3).CX(2,1)
circuit.CX(0,2).CX(3,1)
gadget = quasar.Circuit().Ry(0).CX(0,1)
circuit.add_gate(gadget, (0,1), ascii_symbols=['A0', 'A1'])
circuit.add_gate(gadget, (3,2), ascii_symbols=['B0', 'B1'])
circuit.add_gate(gadget, (0,3), ascii_symbols=['A0', 'A1'])
circuit.add_gate(gadget, (2,1), ascii_symbols=['B0', 'B1'])
circuit.add_gate(gadget, (0,2), ascii_symbols=['A0', 'A1'])
circuit.add_gate(gadget, (3,1), ascii_symbols=['B0', 'B1'])
circuit.add_controlled_gate(gadget, (4,0,1), ascii_symbols=['A0', 'A1'])
circuit.add_controlled_gate(gadget, (5,3,2), ascii_symbols=['B0', 'B1'])
circuit.add_controlled_gate(gadget, (4,0,3), ascii_symbols=['A0', 'A1'])
circuit.add_controlled_gate(gadget, (5,2,1), ascii_symbols=['B0', 'B1'])
circuit.add_controlled_gate(gadget, (4,0,2), ascii_symbols=['A0', 'A1'])
circuit.add_controlled_gate(gadget, (5,3,1), ascii_symbols=['B0', 'B1'])
import time
start = time.time()
print(circuit)
print('%11.3E' % (time.time() - start))

T  : |0|1|2  |3  |4 |5 |6    |7    |8    |9    |10   |11   |12   |13   |14   |15   |

q0 : -H-@-@---@---A0=A0-A0====A0----A0====A0----A0====A0----A0====A0----A0====A0----
        | |   |   |  |  |     |     |     |     |     |     |     |     |     |     
q1 : -X-X-|-X-|-X-A1=A1-|--B1=|==B1-|--B1=|==B1-A1====A1----|--B1=|==B1-|--B1=|==B1-
          | | | |       |  |  |  |  |  |  |  |  |     |     |  |  |  |  |  |  |  |  
q2 : -Y-@-|-@-X-|-B1=B1-|--B0=|==B0-A1=|==A1-|--|--B1=|==B1-|--B0=|==B0-A1=|==A1-|--
        | |     | |  |  |     |        |     |  |  |  |  |  |  |  |  |  |  |  |  |  
q3 : -Z-X-X-----@-B0=B0-A1====A1-------B0====B0-|--B0=|==B0-A1=|==A1-|--|--B0=|==B0-
                                                |  |  |  |  |  |  |  |  |  |  |  |  
q4 : -------------------------------------------@==|==@--|--@==|==@--|--@==|==@--|--
                                                   |     |     |     |     |     |  
q5 : ----------------------------------------------@=====@-----@

In [3]:
print(quasar.Circuit().X(0).X(0).X(0).X(0).X(0).X(0).X(0, time_start=10).X(0).X(0).X(0).X(0).X(0).X(0).X(0))

T  : |0|1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|

q0 : -X-X-X-X-X-X---------X--X--X--X--X--X--X--X--
                                                  
T  : |0|1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|



In [4]:
print(quasar.Circuit())

T  : |

T  : |



In [5]:
def ascii_diagram2(self):
    
    class GateLayout(object):
        
        def __init__(
            self,
            min_qubit=0,
            max_qubit=-1,
            ascii_symbols=[], # None to indicate vertical connector
            ):
        
            self.min_qubit = min_qubit
            self.max_qubit = max_qubit
            self.ascii_symbols = ascii_symbols
            
        @property
        def ascii_symbol_enumeration(self):
            return [(qubit, symbol) for qubit, symbol in zip(range(self.min_qubit, self.max_qubit+1), self.ascii_symbols)]
            
        @property
        def max_ascii_width(self):
            return max(len(_) for _ in self.ascii_symbols if _ is not None)
        
        @staticmethod
        def interferes(layout1, layout2):
            if layout1.min_qubit > layout2.max_qubit: return False
            if layout1.max_qubit < layout2.min_qubit: return False
            return True
            
        @staticmethod
        def build(qubits, ascii_symbols):
            min_qubit = min(qubits)
            max_qubit = max(qubits)
            nqubit = max_qubit - min_qubit + 1
            ascii_symbols2 = [None] * nqubit
            for qubit, ascii_symbol in zip(qubits, ascii_symbols):
                ascii_symbols2[qubit - min_qubit] = ascii_symbol
            return GateLayout(
                min_qubit=min_qubit,
                max_qubit=max_qubit,
                ascii_symbols=ascii_symbols2,
                )
        
    # => Logical Layout <= #
        
    # Map of time : list of [list of GateLayout]
    # Portion in [] represents a relative second within the time index
    layouts = { time : [[]] for time in range(self.min_time, self.max_time+1) }
    # List of (starting second, ending second) for all multi-time gates
    time_connector_keys = []
    # GateLayouts for all multi-time gates
    time_connector_layouts = []
    for key, gate in self.gates.items():
        times, qubits = key
        layout = GateLayout.build(qubits, gate.ascii_symbols)
        time_connector_key = []
        for time in times:
            found = False
            for second, layouts2 in enumerate(layouts[time]):
                if any(GateLayout.interferes(layout, _) for _ in layouts2): 
                    continue
                layouts2.append(layout)
                found = True
                break
            if not found:
                second += 1
                layouts[time].append([layout])
            time_connector_key.append((time, second))
        if len(times) > 1:
            time_connector_keys.append((time_connector_key[0], time_connector_key[-1]))
            time_connector_layouts.append(layout)
                
    # List of tuple of (time, relative_second) for all absolute seconds
    seconds = []
    for time, second_layouts in layouts.items():
        seconds += [(time, _) for _ in range(len(second_layouts))]
        
    # Reverse map of (time, relative_second) : absolute second
    seconds_map = { v : k for k, v in enumerate(seconds) }
    
    # => ASCII Sizes <= #
    
    # Determine ASCII widths of each second
    seconds_ascii_widths = [max(_.max_ascii_width for _ in layouts[time][second]) for time, second in seconds]
    
    # Adjust ASCII widths for time width
    time_widths_header = { time : len(str(time)) for time in range(self.min_time, self.max_time+1)}
    time_widths = { time : 0 for time in range(self.min_time, self.max_time+1)}
    for second_index, key in enumerate(seconds):
        time, second = key
        time_widths[time] += seconds_ascii_widths[second_index]
    for time, second_layouts in layouts.items():
        if len(second_layouts) > 0: time_widths[time] += len(second_layouts) - 1
    time_adjustments = { time : max(time_widths_header[time] - time_widths[time], 0) 
        for time in time_widths.keys()}
    for time, second_layouts in layouts.items():
        if len(second_layouts) == 0: continue
        second = len(second_layouts)-1
        second_index = seconds_map[(time, second)]
        seconds_ascii_widths[second_index] += time_adjustments[time]
    
    # Adjust ASCII widths for separation characters
    seconds_ascii_widths = [_ + 1 for _ in seconds_ascii_widths]
    
    # Determine ASCII starts of each second
    seconds_ascii_starts = [0]
    for index, width in enumerate(seconds_ascii_widths):
        seconds_ascii_starts.append(seconds_ascii_starts[index] + width)
    ascii_width = seconds_ascii_starts[-1] # Total width
    seconds_ascii_starts = seconds_ascii_starts[:-1]
    
    # => ASCII Art <= #
    
    nqubit = self.nqubit
    min_qubit = self.min_qubit
    
    wire_lines = [['-'] * ascii_width for _ in range(nqubit)]
    join_lines = [[' '] * ascii_width for _ in range(nqubit)]
    
    for time_connector_key, time_connector_layout in zip(time_connector_keys, time_connector_layouts):
        second1 = time_connector_key[0]
        second2 = time_connector_key[1]
        second1_index = seconds_map[second1]
        second2_index = seconds_map[second2]
        second1_start = seconds_ascii_starts[second1_index]
        second2_start = seconds_ascii_starts[second2_index]
        for qubit, symbol in time_connector_layout.ascii_symbol_enumeration:
            if symbol is None: continue
            wire_lines[qubit - min_qubit][second1_start:second2_start] = '=' * (second2_start - second1_start)
    
    for time, second_layouts in layouts.items():
        for second, layouts2 in enumerate(second_layouts):
            second_index = seconds_map[(time, second)]
            second_start = seconds_ascii_starts[second_index]
            for layout in layouts2:
                for qubit in range(layout.min_qubit, layout.max_qubit+1):
                    wire_lines[qubit - min_qubit][second_start] = '|'
                for qubit in range(layout.min_qubit, layout.max_qubit):
                    join_lines[qubit - min_qubit][second_start] = '|'
                for qubit, symbol in layout.ascii_symbol_enumeration:
                    if symbol is None: continue
                    for index, char in enumerate(symbol):
                        wire_lines[qubit - min_qubit][second_start + index] = char      
    
    wire_strs = [''.join(_) for _ in wire_lines]
    join_strs = [''.join(_) for _ in join_lines]
    
    qwidth = max(len(str(index - min_qubit)) for index in range(nqubit))
    
    wire_strs = ['q%-*d : -%s\n' % (
        qwidth, 
        index - min_qubit, 
        wire_str,
        ) for index, wire_str in enumerate(wire_strs)]
    
    join_strs = [' %-*s    %s\n' % (
        qwidth, 
        ' ', 
        join_str,
        ) for index, join_str in enumerate(join_strs)]
    
    ascii_str = ''.join(a + b for a, b in zip(wire_strs, join_strs))
    
    time_str = 'T%-*s : |%s\n' % (
        qwidth, 
        ' ',
        ''.join(['%-*d|' % (v, k) for k, v in time_widths.items()])
        )
    
    return time_str + '\n' + ascii_str + time_str

import time
start = time.time()
print(ascii_diagram2(circuit))
print('%11.3E' % (time.time() - start))

T  : |0|1|2  |3  |4 |5 |6    |7    |8    |9    |10   |11   |12   |13   |14   |15   |

q0 : -H-@-@---@---A0=A0-A0====A0----A0====A0----A0====A0----A0====A0----A0====A0----
        | |   |   |  |  |     |     |     |     |     |     |     |     |     |     
q1 : -X-X-|-X-|-X-A1=A1-|--B1=|==B1-|--B1=|==B1-A1====A1----|--B1=|==B1-|--B1=|==B1-
          | | | |       |  |  |  |  |  |  |  |  |     |     |  |  |  |  |  |  |  |  
q2 : -Y-@-|-@-X-|-B1=B1-|--B0=|==B0-A1=|==A1-|--|--B1=|==B1-|--B0=|==B0-A1=|==A1-|--
        | |     | |  |  |     |        |     |  |  |  |  |  |  |  |  |  |  |  |  |  
q3 : -Z-X-X-----@-B0=B0-A1====A1-------B0====B0-|--B0=|==B0-A1=|==A1-|--|--B0=|==B0-
                                                |  |  |  |  |  |  |  |  |  |  |  |  
q4 : -------------------------------------------@==|==@--|--@==|==@--|--@==|==@--|--
                                                   |     |     |     |     |     |  
q5 : ----------------------------------------------@=====@-----@